# Cifar-100 classification via Transfer learning.

in this notebook, we will build and train a transfer learning model on Cifar100 dataset

  

## phase one:
### load libraries

In [ ]:
from google.colab import drive

drive.mount("/content/drive")
%cd /content/drive/MyDrive

In [ ]:
# Load librarys and the TensorBoard notebook extension

import keras
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

%load_ext tensorboard

# clean log dir
!rm -rf logs/

## phase two:
### load data

In [ ]:
# load data and assert the correct number for data
## load Cifar100
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()


# make sure about shape of data
assert x_train.shape == (50000, 32, 32, 3)
assert y_train.shape == (50000, 1)

assert x_test.shape == (10000, 32, 32, 3)
assert y_test.shape == (10000, 1)

# create val and test set
x_val = x_test[:8000]
x_test = x_test[8000:]
y_val = y_test[:8000]
y_test = y_test[8000:]

# make sure about shape of val and test
assert x_val.shape == (8000, 32, 32, 3)
assert y_val.shape == (8000, 1)

assert x_test.shape == (2000, 32, 32, 3)
assert y_test.shape == (2000, 1)

In [ ]:
# visualize some random pics
rand = np.random.randint(low=0, high=50000, size=4)
f, axarr = plt.subplots(2, 2)
axarr[0, 0].imshow(x_train[rand[0]])
axarr[0, 0].set_title(f"label is {y_train[rand[0]]}")
axarr[0, 1].imshow(x_train[rand[1]])
axarr[0, 1].set_title(f"label is {y_train[rand[1]]}")

axarr[1, 0].imshow(x_train[rand[2]])
axarr[1, 0].set_title(f"label is {y_train[rand[2]]}")

axarr[1, 1].imshow(x_train[rand[3]])
axarr[1, 1].set_title(f"label is {y_train[rand[3]]}")

## phase Three:
### create model.
our model will contain of 3 models:
 1.  we will use panda method.
 2. would use augmention
 3. would use Xception pre-trained
 4. would use Dense for classification

In [ ]:
# augmnt model
augment_model = keras.Sequential(
    [
        keras.layers.RandomFlip(mode="horizontal_and_vertical"),
        keras.layers.RandomRotation(0.15),
        keras.layers.RandomZoom(0.15),
        keras.layers.RandomContrast(0.15),
        keras.layers.Resizing(100, 100),
        keras.layers.Rescaling(1.0 / 255),
    ]
)

# pre-Trained model
xcept = keras.applications.Xception(
    include_top=False,
    weights="imagenet",
    input_shape=(100, 100, 3),
)
# setting train ableing OFF
xcept.trainable = False

# 100 class classifire
classifire = keras.Sequential(
    [
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Dropout(0.4),
        keras.layers.Dense(units=512, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(units=256, activation="relu"),
        keras.layers.Dense(units=100, activation="softmax"),
    ]
)
# model in functial style
inputs = keras.Input(shape=(32, 32, 3))
x = augment_model(inputs)
x = xcept(x, training=False)
outputs = classifire(x)

model = keras.Model(inputs, outputs)

model.summary()

In [ ]:
keras.utils.plot_model(model=model, show_shapes=True)

## phase Four:
### compile and train *Freezed* model

In [ ]:
# Compile  model
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["acc"],
)

In [ ]:
# custom call  back for +90 percent acc
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        """
        Halts the training when the accuravy rise  above 90%

        Args:
            epoch (integer) - index of epoch (required but unused in the function definition below)
            logs (dict) - metric results from the training epoch
        """

        # Check the loss
        if logs["val_acc"] > 80:

            # Stop if threshold is met
            print("\nLoss is lower than 0.4 so cancelling training!")
            self.model.stop_training = True


tensorboard_callback = keras.callbacks.TensorBoard(log_dir="./logs")

In [ ]:
epoch = 200
# Train model
hist = model.fit(
    x=x_train,
    y=y_train,
    batch_size=128,
    validation_data=(x_val, y_val),
    epochs=epoch,
    callbacks=[
        myCallback(),
        keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5),
        tensorboard_callback,
    ],
)

In [ ]:
# save model befor defreeze
model.save("cifar10-tl-befor-defreeze.keras")

## phase Five:
### Train End-to-End
final model will be ready soon!

In [ ]:
# defreeze model and retain
xcept.trainable = True
model.summary()
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["acc"],
)

In [ ]:
# End to End train
## NOTE: TRAIN FEW EPOCH 1~5
epoch = 3
model.fit(
    x=x_train, y=y_train, batch_size=128, validation_data=(x_val, y_val), epochs=epoch
)

## FINAL MODEL

In [ ]:
# Evaluate model performance on the test set
model.evaluate(x=x_test, y=y_test)

In [ ]:
# export model
model.save("cifar100-TL-fine-tuned.keras")